# Генерація заголовків для новин

У цій лабораторній роботі Ви познайомитеся з великими мовними моделями на прикладі задачі генерації заголовків для текстів новинного характеру.
І хоча задача є доволі специфічною, ідентичні підходи можна використовувати для вирішення задач автоматичного реферування (стислого переказу) і перефразування текстів та діалогів, а також зворотного процесу: генерації текстів великої розмірності (книжок, статей, чи навіть програмного коду) за коротким описом.


## Kaggle

Локальна робота з великими мовними моделями потребує значних обчислювальних ресурсів. Саме тому для виконання цієї лабораторної роботи рекомендується використовувати сервіс Kaggle.

**Kaggle** - платформа для змагань з машинного навчання, де користувачі змагаються у створенні найкращих моделей для вирішення широкого спектру задач, запропонованих різноманітними компаніями та організаціями. Окрім того, платформа має схожий на Google Colab функціонал для інтерактивної роботи з Python, а також надає доступ до графічних прискорювачів з лімітом у 9 годин на один сеанс та 30 годин на тиждень загалом.

Дізнатися більше про Kaggle Notebooks можна [тут](https://www.kaggle.com/docs/notebooks#technical-specifications), також детальний опис процесу створення ноутбуку наведено [тут](https://www.datacamp.com/tutorial/tutorial-kaggle-datasets-tutorials-kaggle-notebooks).

## Набір даних

У лабораторній роботі використовується підготовлена та зменшена версія [українського датасету](https://huggingface.co/datasets/FIdo-AI/ua-news), що містить новини та їх заголовки.
Для завантаження датасету у якості вхідних даних для створеного ноутбуку необхідно:
- у секції **Input**, що знаходиться справа у меню налаштувань, натиснути на **Upload** та обрати **New Dataset**
- завантажити архів датасету та вказати `fido-dataset` у якості назви, файл буде автоматично розпаковано та додано до папки `/kaggle/input/`

## Підготовка середовища

Тренування моделі потребує мінімум 16 гігабайт VRAM, тому у налаштуваннях ноутбука (**Settings** -> **Accelerator**)  або у секції **Session options** необхідно обрати графічний прискорювач `GPU P100`. Перевірити наявність графічного прискорювача у поточному сеансі можна за допомогою нижченаведеної команди:

In [1]:
# виводить інформацію про графічний прискорювач та його поточне навантаження 
!nvidia-smi

Tue Jul 30 01:17:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Встановлення залежностей
%pip install -U transformers[sentencepiece]
%pip install -U datasets 
%pip install -U evaluate
%pip install -U accelerate
%pip install -U peft
%pip install together
%pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 73.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 8.6 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/

In [3]:
# Імпорт бібліотек
import os
import json

import torch
import numpy as np
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, LoraConfig, get_peft_model
from datasets import load_from_disk, Dataset
from evaluate import load
from together import Together

2024-07-30 01:20:27.530168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 01:20:27.530272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 01:20:27.664876: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
# Файли вхідних даних доступні лише для читання в каталозі `../input/`.
# Наприклад, виконавши цю команду, ви побачите список усіх файлів у каталозі введення

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Ви можете записати до 20 ГБ у поточний каталог (`/kaggle/working/`), який буде збережено як результуючі дані під час створення версії за допомогою "Save & Run All"
# Ви також можете записати тимчасові файли в `/kaggle/temp/`, але вони не будуть збережені поза поточним сеансом

/kaggle/input/fido-dataset/state.json
/kaggle/input/fido-dataset/dataset_info.json
/kaggle/input/fido-dataset/data-00000-of-00001.arrow
/kaggle/input/fido-dataset/cache-77a8597cc930c391.arrow
/kaggle/input/fido-dataset/cache-f9115c0626f3a666.arrow
/kaggle/input/fido-dataset/cache-40a3a7a65adcc2f5.arrow
/kaggle/input/fido-dataset/cache-d4ffd818802c683f.arrow


In [9]:
# визначення обчислювального пристрою для роботи, cuda використовується для сеансу з обраним графічним прискорювачем
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## 1. Підготовка даних

In [13]:
# перенесення набору даних у робочий каталог
!cp -r "/kaggle/input/fido-dataset" "/kaggle/working/"

In [14]:
# функція для перевірки входження тексту у список
def check_uniques(example, uniques):
    return example['text'] not in uniques

# завантаження даних
dataset = load_from_disk("/kaggle/working/fido-dataset")

# поділ даних на набори для тренування та оцінки
train_test_data = dataset.train_test_split(test_size=0.4, seed=42)

# видалення дублікатів тренувальних даних з набору для оцінки
uniques = set(train_test_data['train']['text'])
train_test_data['test'] = train_test_data['test'].filter(check_uniques, fn_kwargs={"uniques": uniques})
# поділ набору для оцінки на тестовий та валідаційний
dev_test_data = train_test_data['test'].train_test_split(test_size=0.5, seed=42)
train_test_data, dev_test_data

Filter:   0%|          | 0/39021 [00:00<?, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['text', 'input_ids', 'attention_mask'],
         num_rows: 58531
     })
     test: Dataset({
         features: ['text', 'input_ids', 'attention_mask'],
         num_rows: 38761
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['text', 'input_ids', 'attention_mask'],
         num_rows: 19380
     })
     test: Dataset({
         features: ['text', 'input_ids', 'attention_mask'],
         num_rows: 19381
     })
 }))

In [15]:
train_data = train_test_data['train']
eval_data = dev_test_data['train']
test_data = dev_test_data['test']

# видалення дублікатів валідаційних даних з тестового набору
uniques = set(eval_data['text'])
test_data = test_data.filter(check_uniques, fn_kwargs={"uniques": uniques})

Filter:   0%|          | 0/19381 [00:00<?, ? examples/s]

In [16]:
train_data, eval_data, test_data

(Dataset({
     features: ['text', 'input_ids', 'attention_mask'],
     num_rows: 58531
 }),
 Dataset({
     features: ['text', 'input_ids', 'attention_mask'],
     num_rows: 19380
 }),
 Dataset({
     features: ['text', 'input_ids', 'attention_mask'],
     num_rows: 19352
 }))

In [17]:
# для зменшення часу тренування використано лише 3000 прикладів для валідації та 500 для тестування
eval_data = Dataset.from_dict(eval_data[:3000])
test_data = Dataset.from_dict(test_data[:500])
train_data, eval_data, test_data

(Dataset({
     features: ['text', 'input_ids', 'attention_mask'],
     num_rows: 58531
 }),
 Dataset({
     features: ['text', 'input_ids', 'attention_mask'],
     num_rows: 3000
 }),
 Dataset({
     features: ['text', 'input_ids', 'attention_mask'],
     num_rows: 500
 }))

In [22]:
train_data[1005:1010]["text"]

["<|endoftext|>Текст: Компанія Xiaomi готує друге покоління ігрового смартфона Black Shark, який з'явиться у продажу вже навесні. Смартфон працює на процесорі Snapdragon 855 з 8 ГБ оперативної пам'яті і Android Pie, пише GSMArena . Ймовірно, смартфон отримає ім'я Black Shark 2, хоча назва поки офіційно не підтверджена. Гаджет повторює дизайн першого Black Shark і наступника Black Shark Helo - темний корпус з вузькими зеленими лініями. На шпигунському знімку пристрою видно, що подвійна камера буде розташована у верхньому лівому кутку. Схоже, що в ній ширококутний об'єктив буде розташований над звичайним. Втім, цілком можливо, що перед нами прототип і все ще може змінитися. Нещодавно Xiaomi представила 27-ватну швидку зарядку для Mi 9 - ймовірно, вона з'явиться і в Black Shark 2. Нещодавно стало відомо, що новий Black Shark з номером моделі SKW-A0 буде комплектуватися зарядним пристроєм MDY-10-EH, який також входить в комплект Mi 9. Новинка буде нести на борту від 8 Гбайт до 12 Гбайт опе

In [37]:
eval_data[45:50]["text"]

['<|endoftext|>Текст: Про це повідомляє Wionews. Землетрус стався в китайському регіоні Сіньцзян на заході країни. В результаті були пошкоджені дахи, сотні жителів були евакуйовані. Після землетрусу залізничне сполучення між столицею регіону Урумчі, Хотан та Аксу було затримано. Китай регулярно потерпає від землетрусів, особливо в його гірських західних й південно-західних регіонах.\n Назва: У Китаї стався потужний землетрус, є загиблі<|endoftext|>',
 '<|endoftext|>Текст: Про це повідомляє прес-служба МОЗ. У зв\'язку з цим, міністерство рекомендує якнайшвидше зробити щеплення проти дифтерії і дорослим, і дітям. "Якісна і безпечна вакцина є в усіх регіонах України, і надається безоплатно як для дітей, так і для дорослих", - йдеться у повідомленні. Зазначається, що рівень вакцинації проти дифтерії лишається низьким - лише половина українських дітей отримала захист третьою дозою вакцини АКДП у віці 18 міс. минулого року, і менше половини дорослих. Дифтерія небезпечна тим, що без негайного

In [40]:
test_data[55:60]["text"]

['<|endoftext|>Текст: Про це повідомляє CNN з посиланням на пресслужбу британського прем’єра. "У заяві Даунінг-стріт зазначено, що уряд має намір закінчити поточну сесію парламенту 8 жовтня. Нове засідання розпочнеться через шість днів, 14 жовтня, з виступу королеви, в якому буде визначено законодавчий порядок денний уряду", - йдеться у повідомленні. У заяві наголошується, що виступ королеви 14 жовтня дозволить уряду "викласти свої плани щодо системи охорони здоров’я, шкіл, боротьби зі злочинністю, інвестування в інфраструктуру та побудови сильної економіки". Тому, як зазначається, робота парламенту має бути призупинена протягом найкоротшого часу, щоб забезпечити всі необхідні матеріально-технічні приготування. Нагадаємо, Верховний суд Великої Британії визнав незаконним рішення прем\'єр-міністра Бориса Джонсона про припинення роботи парламенту на п’ять тижнів.\n Назва: Джонсон знову хоче призупинити роботу парламенту<|endoftext|>',
 '<|endoftext|>Текст: Призначення німця Фелікса Бриха,

## 2. Тренування мовної моделі

Для вирішення задачі генерації заголовків буде використано [мовну модель GPT-2](https://huggingface.co/benjamin/gpt2-large-wechsel-ukrainian), а саме її `large`-версію з 876 мільйонами параметрів, що була попередньо натренована на 40 гігабайтах текстових даних та адаптована для української мови.

GPT - сімейство генеративних мереж на базі архітектури Transformer, що містять тільки шари декодування (Decoder). Процес генерації тексту полягає у тому, що на кожному кроці генерації модель використовує увесь попередній текст (в тому числі і результати попередніх генерацій) для визначення найбільш імовірного наступного слова.

Детальний огляд архітектури Transformer представлено [тут](https://jalammar.github.io/illustrated-transformer/), а з GPT можна ознайомитися [тут](https://jalammar.github.io/illustrated-gpt2/).

<center>
    <figure>
        <img src="https://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif" alt ="Concrete">
        <figcaption>
            <a href="https://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif">Illustrated GPT-2 by Jay Alammar</a></figcaption>
    </figure>
</center>

<center>
    <figure>
        <img src="https://jalammar.github.io/images/gpt2/gpt2-self-attention-example-2.png" alt ="Concrete">
        <figcaption>
            <a href="https://jalammar.github.io/images/gpt2/gpt2-self-attention-example-2.png">Illustrated GPT-2 by Jay Alammar</a></figcaption>
    </figure>
</center>

<center>
    <figure>
        <img src="https://jalammar.github.io/images/gpt2/gpt2-output.png" alt ="Concrete">
        <figcaption>
            <a href="https://jalammar.github.io/images/gpt2/gpt2-output.png">Illustrated GPT-2 by Jay Alammar</a></figcaption>
    </figure>
</center>

### 2.1 Підготовка моделі

In [41]:
hf_model_name = "benjamin/gpt2-large-wechsel-ukrainian"

# завантаження токенізатора
tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
tokenizer

tokenizer_config.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.95M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

GPT2TokenizerFast(name_or_path='benjamin/gpt2-large-wechsel-ukrainian', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [42]:
# Зазвичай текстові дані мають різну довжину, але процедура тренування потребує
# однакової довжини вхідних даних для пакетної (batch) обробки - декілька прикладів
# об'єднуються в один пакет і обробляються одночасно.
# Якщо приклад занадто короткий, то до нього додається необхідна кількість `pad_token` токенів.
# Під час попереднього тренування GPT не використовувалися `pad_token`, так як
# текстовий корпус ділився на блоки одного розміру. В той же час, для нашої задачі
# тексти новин мають різну довжину, тому необхідно додати цей спеціальний токен.
# Також, щоб не проводити тренування нового токена, буде використано спеціальний токен
# <|endoftext|>, що позначає і початок (bos_token) і кінець (eos_token) послідовності.
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.pad_token, tokenizer.eos_token, tokenizer.bos_token)
print(tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.bos_token_id)

<|endoftext|> <|endoftext|> <|endoftext|>
0 0 0


In [43]:
# завантаження моделі
model = AutoModelForCausalLM.from_pretrained(hf_model_name)
# розміщення моделі на графічному прискорювачі
model.to(device)

config.json:   0%|          | 0.00/922 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.39G [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [44]:
model.device

device(type='cuda', index=0)

### 2.2 Генерація

На кожному кроці генерації модель за допомогою функції Softmax будує розподіл ймовірностей, де кожен елемент представляє собою відповідне слово чи токен зі словника моделі.
Елемент з найбільшою ймовірністю обирається у якості найбільш ймовірного продовження тексту з урахуванням усіх попередніх слів. Цей процес вибору має декілька важливих параметрів, одним з яких є температура. Чим більша температура, тим менша відстань від наймовірнішого слова до найменш імовірного, і навпаки. На практиці це означає, що більша температура розширює набір елементів, з яких обирається наступне слово/токен, і модель більше "фантазує". В той же час, зменшення температури призводить до більшої детермінованості, але і генерації будуть однотипними.

<center>
    <figure>
        <img src="https://lena-voita.github.io/resources/lectures/lang_models/sampling/softmax_temperature-min.png" alt ="Concrete">
        <figcaption>
            <a href="https://lena-voita.github.io/resources/lectures/lang_models/sampling/softmax_temperature-min.png">NLP Course by Lena Voita</a></figcaption>
    </figure>
</center>



<center>
    <figure>
        <img src="https://lena-voita.github.io/resources/lectures/lang_models/sampling/temp_diversity_coherence-min.png" alt ="Concrete">
        <figcaption>
            <a href="https://lena-voita.github.io/resources/lectures/lang_models/sampling/temp_diversity_coherence-min.png">NLP Course by Lena Voita</a></figcaption>
    </figure>
</center>

In [45]:
# функція для генерації заголовків
def predict_title(model, inputs: list, postprocess=False, temperature=0.6, max_new_tokens=48) -> list:
    outputs = []
    # з no_grad() градієнти не будуть обчислюватися
    with torch.no_grad():
        for idx, row in enumerate(inputs):
            if (idx+1) % 100 == 0:
                print(f"Generated {idx+1} titles\n")
            # текст містить і новину і назву, необхідно видалити назву
            prompt = row.split("\n Назва:")[0] + "\n Назва:"
            batch = tokenizer([prompt], return_tensors='pt').to(device)
            output_tokens = model.generate(
                **batch,
                max_new_tokens=max_new_tokens, # визначає максимальну кількість згенерованих токенів
                do_sample=True, # семплювання замість жадібного декодування
                temperature=temperature,
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
            # декодування згенерованих ідентифікаторів токенів у текст
            output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
            if postprocess:
                # генеративні моделі без інструкційного налаштування досить часто генерують поки не
                # вичерпано ліміт токенів шляхом повторення вже згенерованого тексту, тому необхідно
                # обрати лише перший результат.
                output = output.split("\n Назва:")[1]
            outputs.append(output.strip())
    return outputs

In [46]:
# генерація заголовків для перших 50 текстів з тестового набору
predicted_titles = predict_title(model, test_data[:50]['text'])

for row in predicted_titles:
    print(row, '\n\n')

Текст: Глава правління Укрпошти Ігор Смілянський сказав в інтерв'ю проекту KRYM, що його місячний оклад збільшився в 2018 році і становить 748 тис. грн. "Місячний оклад - 748 тис. грн. Більшість наших співробітників вважає, що я повинен отримувати 40 тис. Це 10 зарплат листонош", - зазначив він. Смілянський нагадав, що у нього три вищі освіти в американських вузах, і підкреслив, що може відзвітувати за всі свої доходи, на відміну від критикуючих його за високу зарплату депутатів, які "все життя в політиці, але у них будинок в Конча-Заспі і Mercedes". "Я розумію джерело цього. Я знаю, що більшості людей простіше сприймати депутата, який офіційно отримує $300, живе в Конча-Заспі в триповерховому будинку і їздить на Mercedes... Вони знають, що він вкрав, але скільки точно, не знають, а моя зарплата відома", - додав глава Укрпошти. За словами Смілянського, після його приходу в компанію вона почала прозоро проводити держзакупівлі. "Поки ще ніхто не зміг перебити ті ціни, за якими ми купуємо

Згенеровані заголовки здебільшого демонструють відсутність зв'язку з наданим текстом новини, що цілком природно для моделі без достатного тренування під нашу задачу.

### 2.3 Налаштування моделі під задачу

Повноцінне тренування моделі на обраному датасеті потребує як мінімум 20 гігабайт VRAM (P100 має лише 16) та багато часу: в залежності від гіперпараметрів (кількості епох, розміру пакетів та максимальної довжини вхідної послідовності) процедура тренування може потребувати одну або декілька діб.

Для прискорення тренування скористаємося однією доволі поширеною технікою PEFT (Parameter-Efficient Fine-Tuning).

In [47]:
# заморожування параметрів моделі, щоб для них не обчислювалися градієнти під час тренування
for param in model.parameters():
    param.requires_grad = False

### LoRA (Low-Rank Adaptation)

LoRA є одною з технік ефективного налаштування великих моделей без значних потреб у обчислювальних ресурсах.
Метод полягає у тому, що під час тренування параметри моделі не змінюються, а замість цього оновлюються лише адаптери.
Адаптери представляють собою декомпозицію матриці у вигляді добутку двох низькорангових матриць:

- якщо матриця параметрів `W` має розмір `DxD`, то її можна представити як добуток матриць `A` розмірністю `DxR` та `B` розмірністю `RxD`
- якщо ранг `R` набагато менший за `D`, то обчислення відбуватимуться значно швидше
- під час прямого поширення заморожені ваги моделі множаться на вхідний сигнал `X`, аналогічні дії виконуються і з адаптерами, а далі результати обох операцій додаються
- під час зворотного поширення оновлюються лише параметри адаптерів.

Детальніше ознайомитися з LoRA можна за [посиланням](https://www.datacamp.com/tutorial/mastering-low-rank-adaptation-lora-enhancing-large-language-models-for-efficient-adaptation).

<center>
    <figure>
        <img src="https://www.unite.ai/wp-content/uploads/2023/10/lora-animated.gif" alt ="Concrete">
        <figcaption>
            <a href="https://www.unite.ai/wp-content/uploads/2023/10/lora-animated.gif">LoRA by unite.ai</a></figcaption>
    </figure>
</center>

In [48]:
# конфігурація LoRa
# перелік усіх доступних параметрів: https://huggingface.co/docs/peft/package_reference/lora
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [49]:
# синтез нової моделі на базі підготовленої та конфігурації LoRA
model = get_peft_model(model, config)

model.device

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


device(type='cuda', index=0)

In [50]:
def print_trainable_parameters(model):
    """
    Обчислення кількості параметрів моделі, що будуть оновлюватися під час тренування
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 2949120 || all params: 776979200 || trainable%: 0.3795622842928099


In [51]:
# очищення оперативної пам'яті (опціонально)
import gc
gc.collect()

44

#### Варіанти для тренування моделі

Варіант = Номер_у_списку % 4 + 1

|**№**| Max steps | Warmup steps | Eval steps | Save steps | 
|-----|-----------|--------------|------------|------------|
|**1**| 2700      | 50           | 270        | 270        | 
|**2**| 2800      | 100          | 280        | 280        |
|**3**| 2900      | 150          | 290        | 290        |
|**4**| 3100      | 200          | 310        | 310        |

In [53]:
# конфігурація тренера
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, # розмір пакету для тренування
        per_device_eval_batch_size=4, # розмір пакету для валідації
        gradient_accumulation_steps=4, # https://discuss.huggingface.co/t/batch-size-vs-gradient-accumulation/5260/6
        warmup_steps=100, # перші N тренувальних кроків learning_rate буде поступово зростати до заданого значення, щоб уникнути упередженість через порядок прикладів
        max_steps=3000, # загальна кількість тренувальних кроків
        learning_rate=2e-4,
        logging_steps=25,
        output_dir='outputs_gpt2large_title_generation_lr2e04',
        do_eval=True,
        eval_strategy="steps",
        eval_steps=300, # валідація відбувається кожні N тренувальних кроків
        save_steps=300, # параметри моделі зберігаються кожні N тренувальних кроків
        fp16=True, # тренування зі змішаною точністю дозволяє збільшити швидкість процесу
        bf16=False,
        #auto_find_batch_size=True, знаходить оптимальний розмір пакету у випадку OOM помилок
        #use_cpu=True, причини помилок під час тренування набагато легше визначати перезапуском процедури на CPU
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

max_steps is given, it will override any value given in num_train_epochs


In [54]:
# тренування
# важливо: для запуску тренування у Kaggle необхідно виконати інструкції wandb, що будуть виведені на екран після запуску команди
model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
300,2.456200,2.372811
600,2.400000,2.354584
900,2.408300,2.346055
1200,2.391900,2.338896
1500,2.441100,2.334228
1800,2.420800,2.330592
2100,2.409900,2.327605
2400,2.401700,2.325484
2700,2.413500,2.324033
3000,2.390400,2.323658


TrainOutput(global_step=3000, training_loss=2.4227334175109863, metrics={'train_runtime': 14162.8091, 'train_samples_per_second': 3.389, 'train_steps_per_second': 0.212, 'total_flos': 6.943753244055552e+16, 'train_loss': 2.4227334175109863, 'epoch': 0.8200642383653386})

In [55]:
# збереження моделі після завершення тренування
torch.save(model.state_dict(), '/kaggle/working/gpt2large_title_generation_lr2e04_step3000_lora.pt')

## 3. Тестування моделі

In [58]:
# завантаження моделі з контрольної точки
adapter_checkpoint_path = "/kaggle/working/outputs_gpt2large_title_generation_lr2e04/checkpoint-3000/"

base_model = AutoModelForCausalLM.from_pretrained(
    hf_model_name,
    device_map=device,
)

model = PeftModel.from_pretrained(base_model, adapter_checkpoint_path, device_map=device)
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1280)
        (wpe): Embedding(1024, 1280)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-35): 36 x GPT2Block(
            (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3840, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
               

#### Варіанти для генерації заголовків

Варіант = Номер_у_списку % 4 + 1

|**№**| Temperature |
|-----|-------------|
|**1**| 0.5         |
|**2**| 0.55        |
|**3**| 0.6         |
|**4**| 0.65        |

In [102]:
# генерація заголовків для перших 10 текстів з тестового набору
predicted_titles = predict_title(model, test_data[:10]['text'], temperature=0.5)

for row in predicted_titles:
    print(row, '\n\n')

Текст: Глава правління Укрпошти Ігор Смілянський сказав в інтерв'ю проекту KRYM, що його місячний оклад збільшився в 2018 році і становить 748 тис. грн. "Місячний оклад - 748 тис. грн. Більшість наших співробітників вважає, що я повинен отримувати 40 тис. Це 10 зарплат листонош", - зазначив він. Смілянський нагадав, що у нього три вищі освіти в американських вузах, і підкреслив, що може відзвітувати за всі свої доходи, на відміну від критикуючих його за високу зарплату депутатів, які "все життя в політиці, але у них будинок в Конча-Заспі і Mercedes". "Я розумію джерело цього. Я знаю, що більшості людей простіше сприймати депутата, який офіційно отримує $300, живе в Конча-Заспі в триповерховому будинку і їздить на Mercedes... Вони знають, що він вкрав, але скільки точно, не знають, а моя зарплата відома", - додав глава Укрпошти. За словами Смілянського, після його приходу в компанію вона почала прозоро проводити держзакупівлі. "Поки ще ніхто не зміг перебити ті ціни, за якими ми купуємо

In [103]:
# генерація заголовків для усього тестового набору
predicted_titles = predict_title(model, test_data['text'], postprocess=True, temperature=0.5)

for idx, row in enumerate(predicted_titles[:50]):
    print(test_data['text'][idx], "\n", "Згенерована назва:", row, '\n\n')

Generated 100 titles

Generated 200 titles

Generated 300 titles

Generated 400 titles

Generated 500 titles

<|endoftext|>Текст: Глава правління Укрпошти Ігор Смілянський сказав в інтерв'ю проекту KRYM, що його місячний оклад збільшився в 2018 році і становить 748 тис. грн. "Місячний оклад - 748 тис. грн. Більшість наших співробітників вважає, що я повинен отримувати 40 тис. Це 10 зарплат листонош", - зазначив він. Смілянський нагадав, що у нього три вищі освіти в американських вузах, і підкреслив, що може відзвітувати за всі свої доходи, на відміну від критикуючих його за високу зарплату депутатів, які "все життя в політиці, але у них будинок в Конча-Заспі і Mercedes". "Я розумію джерело цього. Я знаю, що більшості людей простіше сприймати депутата, який офіційно отримує $300, живе в Конча-Заспі в триповерховому будинку і їздить на Mercedes... Вони знають, що він вкрав, але скільки точно, не знають, а моя зарплата відома", - додав глава Укрпошти. За словами Смілянського, після його п

In [104]:
# виділення справжніх заголовків, що були зібрані разом з текстами новин
orig_titles = []
for row in test_data['text']:
    title = row.split("\n Назва: ")[1].replace("<|endoftext|>", "")
    orig_titles.append(title)
    print(title)

Глава Укрпошти: Моя зарплата - 748 тис. грн. Більшість співробітників вважає, що я повинен отримувати 40 тис.
Кількість загиблих через землетрус у Мексиці зросла до 295 людей
Сутички під Нацполіцією на Подолі: поліція оголосила 4 підозри за штурм відділку
Залишають неідентифікованими 762 загиблих в АТО бійця
20 липня на фронті: з українського боку - 2 поранених, з російського - 6
Тамаш КАДАР: «Майже всі гравці Динамо здорові»
Тука пояснив, чим загрожує невикачування води з шахт на Донбасі
Бойовий комар. У США створили дрон вагою всього чверть грама
Гройсман презентував "митну сотню"
Німеччина планує посилити свою армію додатковими БТРами через путінську агресію
"Айтішники"-шахраї вкрали у банку майже 1,5 млн грн під виглядом перевірки платіжної системи
На 50% дешевше за звичайні. У США продається перший будинок, надрукований на 3D-принтері — фото
Хемілтон здивований поведінкою Ферстаппена після аварії
Україна має шанс. Без США: друга збірна відмовилася від участі в ЧС-2021
Ефективний а

In [105]:
len(predicted_titles), len(orig_titles)

(500, 500)

In [106]:
predicted_titles

['Смілянський розповів, що його зарплата зросла в 2018 році в два рази і становить 748 тис. грн.',
 'В Мексиці під завалами будинків знайшли ще 153 загиблих, - ЗМІ. Врятовано більше 100 людей, - ЗМІ',
 'Поліція затримала радикалів, які напали на поліцейських під Подільським управлінням поліції у Києві, - Князєв. ФОТОрепортаж',
 'У бойовиків є ДНК-коди загиблих на Донбасі українців, - Богомолець про небезпеку для здоров\'я громадян, що живуть по той бік кордону, - "112 Україна"',
 'Бойовики стріляли з гранатометів і кулеметів, поранено двох українських військових, - Міноборони України про ситуацію в зоні ООС 21 липня, - штаб ОС, - відео (оновлено)',
 'Тамаш КАДАР: «Відчуваю себе чудово» в матчі з Шахтарем за Суперкубок України (ФОТО) ⋆ ВІДЕО та ФОТО | Динамо Київ від Шурика ⋆',
 'У МінТОТ розповіли про небезпеку підтоплень на Донбасі через танення снігу та танення льодовиків, - Тука',
 'У Гарварді створили найменший дрон у світі. Відео. Фото та опис проекту. ВIДЕО',
 'Гройсман анонсував

In [107]:
# збереження згенерованих заголовків
with open("/kaggle/working/generated_by_gpt.json", "w", encoding='utf-8') as fw:
    json.dump(predicted_titles, fw)

In [108]:
with open("/kaggle/working/generated_by_gpt.json", "r", encoding='utf-8') as fr:
    predicted_titles = json.load(fr)
predicted_titles

['Смілянський розповів, що його зарплата зросла в 2018 році в два рази і становить 748 тис. грн.',
 'В Мексиці під завалами будинків знайшли ще 153 загиблих, - ЗМІ. Врятовано більше 100 людей, - ЗМІ',
 'Поліція затримала радикалів, які напали на поліцейських під Подільським управлінням поліції у Києві, - Князєв. ФОТОрепортаж',
 'У бойовиків є ДНК-коди загиблих на Донбасі українців, - Богомолець про небезпеку для здоров\'я громадян, що живуть по той бік кордону, - "112 Україна"',
 'Бойовики стріляли з гранатометів і кулеметів, поранено двох українських військових, - Міноборони України про ситуацію в зоні ООС 21 липня, - штаб ОС, - відео (оновлено)',
 'Тамаш КАДАР: «Відчуваю себе чудово» в матчі з Шахтарем за Суперкубок України (ФОТО) ⋆ ВІДЕО та ФОТО | Динамо Київ від Шурика ⋆',
 'У МінТОТ розповіли про небезпеку підтоплень на Донбасі через танення снігу та танення льодовиків, - Тука',
 'У Гарварді створили найменший дрон у світі. Відео. Фото та опис проекту. ВIДЕО',
 'Гройсман анонсував

### 3.1 Оцінка точності моделі

Задачі генерації контенту мають одне суттєве обмеження - для оцінки ефективності побудованих рішень необхідно встановлювати власні метрики та проводити оцінювання з залученням людей.
Якщо для задач класифікації простір правильних відповідей обмежено, то для значної частини задач генерації він майже неперервний, обмежений лише словником мови.

Ручне оцінювання потребує багато ресурсів, тому у цій роботі буде використано один з методів автоматичної оцінки.

#### BERTScore

Метод автоматичної оцінки BERTScore використовує попередньо натреновані векторні представлення моделей на базі архітектури BERT і зіставляє слова у текстах-передбаченнях з правильними за допомогою косинусної подібності. Було продемонстровано, що цей метод корелює з людською оцінкою на рівні речення.

In [109]:
# завантаження метрики
bertscore = load("bertscore")

In [110]:
# оцінка заголовків, що були згенеровані натренованою моделлю
results = bertscore.compute(predictions=predicted_titles, references=orig_titles, model_type="microsoft/mdeberta-v3-base")
{metric: np.mean(results[metric]) for metric in ["precision", "recall", "f1"]}

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


{'precision': 0.6203205051422119,
 'recall': 0.6979209215044976,
 'f1': 0.6555293062329293}

## 4. Тестування великої мовної моделі за допомогою API

У цьому розділі Ви порівняєте результати генерації натренованого рішення з дійсно великою мовною моделлю, що має десятки мільярдів параметрів.
Для цього необхідно зареєструватися на платформі [together.ai](https://together.ai), що надає тарифікований доступ до загальнодоступних мовних моделей через API. Після реєстрації новим користувачам виділяється тестовий баланс у розмірі 5 USD.

Документація платформи доступна за [посиланням](https://docs.together.ai/docs/quickstart).

In [45]:
# скопіюйте свій ключ доступу з https://api.together.ai/settings/api-keys та вставте його нижче
client = Together(api_key="YOUR_API_KEY")

In [60]:
def generate_response(inputs, model, prompt, postprocess=True, max_tokens=512, temperature=0.7):
    outputs = []
    for idx, row in enumerate(inputs):
        if (idx+1) % 100 == 0:
            print(f"Processed {idx+1} texts")
        # видалення заголовку та спеціальних токенів з вхідної послідовності
        row = row.split("\n Назва:")[0].replace("<|endoftext|>", "")
        message = prompt.format(text=row)
        print(message, "\n")
        # взаємодія з моделями відбувається у форматі чату
        # адаптовано з https://api.together.ai/playground/chat/meta-llama/Meta-Llama-3-70B-Instruct-Turbo
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1,
            stop=["<|eot_id|>"], # залежить від сімейства моделей
            stream=False
        )
        output = response.choices[0].message.content
        if postprocess:
            if output.startswith("Назва:"):
                output = output[6:]
        outputs.append(output.strip())
    return outputs

#### Варіанти для генерації заголовків

Варіант = Номер_у_списку % 4 + 1

|**№**| Temperature |
|-----|-------------|
|**1**| 0.5         |
|**2**| 0.6         |
|**3**| 0.7         |
|**4**| 0.8         |

In [63]:
# генерація заголовків для тестового набору моделлю Llama 3.1, що містить 70 мільярдів параметрів та була додатково налаштована інструкціями
model = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
prompt = "Підберіть назву до наведеного нижче тексту.\n{text}\nНазва:"

llama_titles = generate_response(test_data["text"], model, prompt)

Підберіть назву до наведеного нижче тексту.

Текст: Глава правління Укрпошти Ігор Смілянський сказав в інтерв'ю проекту KRYM, що його місячний оклад збільшився в 2018 році і становить 748 тис. грн. "Місячний оклад - 748 тис. грн. Більшість наших співробітників вважає, що я повинен отримувати 40 тис. Це 10 зарплат листонош", - зазначив він. Смілянський нагадав, що у нього три вищі освіти в американських вузах, і підкреслив, що може відзвітувати за всі свої доходи, на відміну від критикуючих його за високу зарплату депутатів, які "все життя в політиці, але у них будинок в Конча-Заспі і Mercedes". "Я розумію джерело цього. Я знаю, що більшості людей простіше сприймати депутата, який офіційно отримує $300, живе в Конча-Заспі в триповерховому будинку і їздить на Mercedes... Вони знають, що він вкрав, але скільки точно, не знають, а моя зарплата відома", - додав глава Укрпошти. За словами Смілянського, після його приходу в компанію вона почала прозоро проводити держзакупівлі. "Поки ще ніхто 

In [64]:
len(llama_titles), llama_titles

(500,
 ['Глава Укрпошти заявив про свій місячний оклад в 748 тисяч гривень',
  'Під завалами будівель у Мексиці знайдено тіла майже 300 осіб.',
  'Підозру оголошено одному з затриманих на акції біля Подільського управління поліції у Києві',
  'На сході України загинуло понад 1 тисячу людей, тіла яких не були ідентифіковані',
  'На Донбасі російські бойовики знову порушили режим припинення вогню.',
  'Кадар про матч з «Шахтарем»: «Віддамо всі сили на полі»',
  'Україна ризикує потрапити в екологічну катастрофу через затоплення шахт на Донбасі',
  '"Найменший дрон у світі з маховими крилами"',
  'Україна запускає мобільні групи для боротьби з контрабандою на митниці',
  'Німеччина планує закупити додатково 131 бронетранспортер Boxer',
  'Кіберполіція викрила групу, яка обманом привласнила 1,4 млн грн банку',
  'Перший будинок, надрукований на 3D принтері, вийшов на ринок нерухомості в США',
  'Хемілтон здивований поведінкою Ферстаппена після аварії',
  'Збірна України з гандболу очікує н

In [65]:
# збереження згенерованих заголовків
with open("generated_by_llama.json", "w", encoding='utf-8') as fw:
    json.dump(llama_titles, fw)

In [67]:
with open("generated_by_llama.json", "r", encoding='utf-8') as fr:
    titles_generated_by_llama = json.load(fr)
len(titles_generated_by_llama), titles_generated_by_llama

(500,
 ['Глава Укрпошти заявив про свій місячний оклад в 748 тисяч гривень',
  'Під завалами будівель у Мексиці знайдено тіла майже 300 осіб.',
  'Підозру оголошено одному з затриманих на акції біля Подільського управління поліції у Києві',
  'На сході України загинуло понад 1 тисячу людей, тіла яких не були ідентифіковані',
  'На Донбасі російські бойовики знову порушили режим припинення вогню.',
  'Кадар про матч з «Шахтарем»: «Віддамо всі сили на полі»',
  'Україна ризикує потрапити в екологічну катастрофу через затоплення шахт на Донбасі',
  '"Найменший дрон у світі з маховими крилами"',
  'Україна запускає мобільні групи для боротьби з контрабандою на митниці',
  'Німеччина планує закупити додатково 131 бронетранспортер Boxer',
  'Кіберполіція викрила групу, яка обманом привласнила 1,4 млн грн банку',
  'Перший будинок, надрукований на 3D принтері, вийшов на ринок нерухомості в США',
  'Хемілтон здивований поведінкою Ферстаппена після аварії',
  'Збірна України з гандболу очікує н

In [68]:
# оцінка заголовків, що були згенеровані моделлю Llama 3.1 70b instruct turbo
results = bertscore.compute(predictions=llama_titles, references=orig_titles, model_type="microsoft/mdeberta-v3-base")
{metric: np.mean(results[metric]) for metric in ["precision", "recall", "f1"]}

{'precision': 0.6964456906318665,
 'recall': 0.6827381086945534,
 'f1': 0.6880039711594581}

## 5.* Спробуйте покращити якість генерацій для моделі з попереднього розділу 